In [1]:
import requests

In [2]:
import math

auth=('admin','supersecret123')

r = requests.get("https://harrigt.papro.org.uk/api/news?hidden=true", auth=auth)
result = r.json()

articles=[]
for i in range(math.ceil(result['total_count']/10)):
    print("Getting page {}".format(i+1))
    offset = i*10
    
    r = requests.get("https://harrigt.papro.org.uk/api/news",
                     params={"hidden":"true",
                            "offset":offset},
                     auth=auth)
    articles.extend(r.json()['articles'])

Getting page 1
Getting page 2
Getting page 3
Getting page 4
Getting page 5
Getting page 6
Getting page 7
Getting page 8
Getting page 9
Getting page 10
Getting page 11
Getting page 12
Getting page 13
Getting page 14


In [3]:
len(articles)

137

In [4]:
import math

r = requests.get("https://harrigt.papro.org.uk/api/news?spam=true", auth=auth)
result = r.json()

spam=[]
for i in range(math.ceil(result['total_count']/10)):
    print("Getting page {}".format(i+1))
    offset = i*10
    
    r = requests.get("https://harrigt.papro.org.uk/api/news",
                     params={"spam":"true",
                            "offset":offset},
                     auth=auth)
    spam.extend(r.json()['articles'])

Getting page 1
Getting page 2
Getting page 3
Getting page 4
Getting page 5
Getting page 6
Getting page 7
Getting page 8
Getting page 9
Getting page 10
Getting page 11
Getting page 12
Getting page 13
Getting page 14
Getting page 15
Getting page 16
Getting page 17
Getting page 18
Getting page 19
Getting page 20
Getting page 21
Getting page 22
Getting page 23
Getting page 24


In [5]:
import math

r = requests.get("https://harrigt.papro.org.uk/api/news?linked=true", auth=auth)
result = r.json()

linked=[]
for i in range(math.ceil(result['total_count']/10)):
    print("Getting page {}".format(i+1))
    offset = i*10
    
    r = requests.get("https://harrigt.papro.org.uk/api/news",
                     params={"linked":"true",
                            "offset":offset},
                     auth=auth)
    linked.extend(r.json()['articles'])

Getting page 1
Getting page 2
Getting page 3
Getting page 4


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()

X = vect.fit_transform( [ article['content'] for article in (articles + linked + spam) ])

In [7]:
y = ["ham"] * (len(articles) + len(linked)) + ['spam'] * len(spam)

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [10]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train.toarray(), y_train)

GaussianNB(priors=None)

In [11]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict

predicted = cross_val_predict(clf, X.toarray(), y)
accuracy_score(predicted, y)

0.74689826302729534

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline

from sklearn import svm

parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__min_df':(0, 0.05, 0.1),
    'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    #'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__C' : [1, 2, 5, 10, 20, 100],
    'clf__kernel' : ['linear', 'rbf']
    #'clf__n_estimators': [10,20,50,100]
}

cv_splits = 5
num_threads=-1

pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', svm.SVC(C=1, probability=True)),
    #('clf', ExtraTreesClassifier())
])

clf = GridSearchCV(pipeline, parameters, verbose=1, n_jobs=num_threads, cv=cv_splits)

In [16]:
X = [ article['content'] for article in (articles + linked + spam) ]

In [ ]:
clf.fit(X,y)

In [15]:
clf.best_params_

{'clf__C': 1,
 'clf__kernel': 'linear',
 'tfidf__norm': 'l2',
 'vect__max_df': 0.75,
 'vect__max_features': 10000,
 'vect__min_df': 0,
 'vect__ngram_range': (1, 2)}

In [27]:
#X_train, X_test, y_train, y_test = train_test_split(x,y)


pipeline = Pipeline([
    #('vect', CountVectorizer(analyzer=stemmed_words)),
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', svm.SVC()),
    #('clf', ExtraTreesClassifier())
])

params = {'clf__C': 1,
 'clf__kernel': 'linear',
 'clf__probability' : True,
 'tfidf__norm': 'l2',
 'vect__max_df': 0.75,
 'vect__max_features': 10000,
 'vect__min_df': 0,
 'vect__ngram_range': (1, 2)}

pipeline.set_params(**params)

#y_pred = pipeline.predict(y_test)

from sklearn.model_selection import cross_val_predict


y_pred = cross_val_predict(pipeline, X,y, cv=5)

In [28]:
from sklearn.metrics import classification_report


print(classification_report(y,y_pred))

             precision    recall  f1-score   support

        ham       0.88      0.82      0.85       169
       spam       0.87      0.92      0.90       234

avg / total       0.88      0.88      0.88       403



In [29]:
import pickle

pipeline.fit(X,y)

with open("spam_classifier.pickle", "wb") as f:
    pickle.dump(pipeline, f)

In [30]:
p = pickle.load(open("spam_classifier.pickle","rb"))

In [36]:
p.predict_proba([""""""])

array([[ 0.00283384,  0.99716616]])

In [39]:
p.classes_

array(['ham', 'spam'],
      dtype='<U4')